In [45]:
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
import pandas as pd
#url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
confirmed = pd.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
dead = pd.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
recovered = pd.read_csv(url, error_bad_lines=False)

In [46]:
confirmed['region']=confirmed['Country/Region'].map(str)+'_'+confirmed['Province/State'].map(str)
dead['region']=dead['Country/Region'].map(str)+'_'+dead['Province/State'].map(str)
recovered['region']=recovered['Country/Region'].map(str)+'_'+recovered['Province/State'].map(str)

In [47]:
newDead = dead.copy()
newConfirmed = confirmed.copy()
newRecovered = recovered.copy()

newColumn = []
for i in dead.columns:
    newColumn.append(i + 'dead')
    
newDead.columns = newColumn

newColumn = []
for i in confirmed.columns:
    newColumn.append(i + 'confirmed')
    
newConfirmed.columns = newColumn


newColumn = []
for i in recovered.columns:
    newColumn.append(i + 'recovered')
    
newRecovered.columns = newColumn


In [48]:
large_dataset = pd.concat([newConfirmed, newRecovered, newDead], axis=1)

In [40]:
import json
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [52]:
for i in range(4, len(confirmed.columns.values)-1):
    geojson_dict = df_to_geojson(large_dataset, properties=[(confirmed.columns.values[i] + 'confirmed'), (confirmed.columns.values[i] + 'dead'), (confirmed.columns.values[i] + 'recovered'), "Latconfirmed", "Longconfirmed"], lat='Latconfirmed', lon="Longconfirmed");
    geojson_str = json.dumps(geojson_dict, indent=2);
    
    
    time = ('0' + confirmed.columns.values[i] + '20');
    time = time.replace('/', '-');

    # save the geojson result to a file
    output_filename = '../data/'+ time + '.geojsonp'
    with open(output_filename, 'w') as output_file:
        output_file.write('eqfeed_callback({})'.format(geojson_str))
    
    

In [7]:
# # Code has already been ran
# for i in range(30, len(confirmed.columns.values)-1):
#     confirmed_dropped = confirmed.drop(confirmed.columns.values[0:2], axis=1).drop(confirmed.columns.values[4:i], axis=1).drop(confirmed.columns.values[i+1:], axis=1);
#     dead_dropped = dead.drop(confirmed.columns.values[0:2], axis=1).drop(confirmed.columns.values[4:i], axis=1).drop(confirmed.columns.values[i+1:], axis=1);
#     recovered_dropped = recovered.drop(confirmed.columns.values[0:2], axis=1).drop(confirmed.columns.values[4:i], axis=1).drop(confirmed.columns.values[i+1:], axis=1);

#     geojson_dict = df_to_geojson(confirmed_dropped, properties=[confirmed.columns.values[i], "Lat", "Long"], lat='Lat', lon="Long");
#     geojson_str = json.dumps(geojson_dict, indent=2);
    
    
#     time = ('0' + confirmed.columns.values[i] + '20');
#     time = time.replace('/', '-');

#     # save the geojson result to a file
#     output_filename = '../data/'+ time + '.geojsonp'
#     with open(output_filename, 'w') as output_file:
#         output_file.write('eqfeed_callback({})'.format(geojson_str))
    
    

KeyError: 'Canada'